<a href="https://colab.research.google.com/github/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/Chapter_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 15. 전이학습
* 전이학습이란?
* 기존의 사전 훈련 방식
* ELMO
* BERT
* OpenAI의 GPT-2

## 15.1 전이학습이란?

* 신경망의 일부 또는 전체 신경망 가중치 파라미터를 MLE로 훈련시키기 전에 다른 데이터셋이나 목적 함수를 사용해 미리 훈련한 후에 본격적인 학습에서 신경망 가중치 파라미터를 더 쉽게 최적화하는 것이다.
* 영상처리와 다르게 자연어 처리 분야는 전이학습보다 본직적인 문제 해결에 집중했다.
  * 현재는 전이학습 방법이 뛰어난 성능을 보이며 주류가 되고 있다.
* 전이학습
  * 더 넓은 범위의 목적을 갖는 비지도학습 문제의 전역 최소점이 더 작은 범위의 목적을 갖는 지도학습 문제의 전역 최소점을 포함할 것이다.
  * 비지도학습을 통해 지역 최소점을 찾으면 추가로 지도학습에서 더 낮은 지역 최소점에 도달할 것이다.
  * 최근 모델이 깊고 넓어서 많은 가중치 파라미터를 가지면 지역 최소점문제가 자연스럽게 해결될 것으로 본다.
  * 랜덤 초깃값에서 최적화 시작 : 데이터셋의 노이즈와 경사하강법 한계
  * 사전 훈련된 가중치 파라미터 값에서 최적화 시작 : 좀더 높은 성능 기대 가능
    * 완전히 다른 성격의 데이터셋이나 문제는 도움이 되지 않을 수 있다.
  * 사전 훈련에 사용된 훈련 데이터셋이 매우 방대하고 자세한 정보를 담는다면, 새로운 문제 위한 데이터셋에 없던 정보를 가질 수 있다.
    * 해결 능력을 추가로 얻게 되고 사전 훈련 덕분에 성능 향상이 가능하다.

## 15.2 기존의 사전 훈련 방식
* word2vec이후 사전 훈련된 단어 임베딩 벡터로 딥러닝 모델 개선
  * 딥러닝 모델에서는 이 방식은 큰 효과가 없다.
* 사전 훈련된 단어 임베딩 벡터를 텍스트 분류와 같은 문제 적용시 방법 소개
  * 기존의 단어 임베딩 벡터 방법을 전이학습에 적용하는 법
  * RNN 텍스트 분류 신경망 가정
    * 1번 데이터셋에서 데이터 x, y를 받아 훈련하는 신경망 
    $\{\theta, W_{emb}, W_{softmax}\}$
    <img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/image1.png?raw=1" width="50%" height="50%">
  * 신경망 : 소프트맥스 계층 + RNN 계층 + 임베딩 계층 구성
  * 원핫 인코딩 벡터들을 문장 내 단어 대신 입력 x로 넣고 신경망 $f_{\theta}$ 통과시켜 클래스 예측하는 분포 $\hat{y}$ 반환하도록 구성한다.  
  <img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/image2.png?raw=1" width="50%" height="50%">  
    * 원핫 인코딩 벡터에서 1 위치 값만 반환
    * 임베딩 계층의 가중치 파라미터 $W$의 각 행 = 해당 단어의 임베딩 벡터



### 15.2.1 사전 훈련된 가중치 파라미터 사용 방법
> 신경망 가중치 파라미터 초깃값으로 사용  

* 각 단어의 임베딩 벡터를 사전 훈련한 단어 임베딩 벡터로 치환하여 사용
$\rightarrow$ 해당 임베딩 계층의 가중치 파라미터 값, 기타 신경망의 랜덤 초기화된 가중치 파라미터 값에 대해 역전파 알고리즘과 경사하강법을 통해 학습 수행

* 0번 데이터셋을 통해 사전 훈련한 단어 임베딩 벡터를 임베딩 계층에 적용
    <img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/image3.png?raw=1" width="50%" height="50%">

    * 각 단어에 해당하는 단어 임베딩 벡터 \
    = 임베딩 계층의 가중치 파라미터의 해당 행을 대체하여 초깃값 설정 가능

> 신경망 가중치 파라미터 값으로 고정

* 0번 데이터셋으로 사전 훈련한 단어 임베딩 벡터를 임베딩 계층에 적용
  * 그러나 이후 훈련에서 임베딩 계층을 제외한 빨간 점선 안쪽 부분의 신경망만 MLE 통해 갱신  
  <img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/image4.png?raw=1" width="50%" height="50%">

  * 임베딩 계층의 가중치 파라미터는 최적화 대상에서 제외
    * 다른 가중치 파라미터만 학습하도록 할 수 있다.
      * 기존에는 경사하강법 수행을 위해 다음처럼 아담 옵티마이저에게 파라미터를 등록해주어야 했다.
      * model.parameters() : 모델 내 가중치 파라미터 반환 이터레이터

            optimizer = optim.Adam(model.parameters())

      * 임베딩 계층을 제외하고 신경망 내 다른 구성요소들의 파라미터들 등록 가능하다.

            optimizer = optim.Adam(model.softmax_layer.parameters() + model.rnn.parameters())

> 신경망 가중치 파라미터 초깃값으로 사용 및 느린 학습  

* 임베딩 계층을 초기화하지만 두 개의 다른 학습률로 임베딩 계층은 빨간색 점선 부분과 달리 **천천히 갱신되게 한다.**  
<img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/image5.png?raw=1" width="50%" height="50%">

* 임베딩 계층의 가중치 파라미터를 최적화에서 제외하는 대신 **천천히 학습하도록 할 수 있다.**
  * 또 다른 학습률이 하이퍼파라미터로 추가된다.

         optimizer = optim.Adam(model.softmax_layer.parameters() + model.rnn.parameters())
         optimizer_emb = optim.Adam(model.emb_layer.parameters(), lr=1e-6)

### 15.2.2 사전 훈련된 단어 임베딩 벡터를 쓰지 않는 이유

  * 기존의 전이학습 방식이 큰 효과가 없는 이유

> 문장의 문맥을 반영하지 못하는 기존 단어 임베딩 알고리즘

* word2vec, Skip-gram, GloVe 알고리즘
  * 성공적으로 단어를 잠재 공간에 임베딩 가능
  * 문장에 함께 출현한 단어 예측에 기반하므로 임베딩된 정보가 매우 한정적
  * 목적 함수는 실제 수행하려는 문제 해결을 위한 목적 함수와 매우 다르다.  
  $\rightarrow$ 특징 반영이 어렵다.
  * 기존 단어 임베딩 알고리즘 결과는 문장의 문맥 고려한 단어 의미를 임베딩하기에 지나치게 단순하다.
    * 같은 단어라도 문장의 문맥에 따라 그 의미가 확연하게 달라지는데 이를 고려하지 못한다.
    * 목적어에 따른 play의 의미


| Play의 의미 | 목적어  |
| :--------- | :------ |
| 연주하다| guitar |
| 재생하다 | movie |
| 놀자 | football |

* 문장 내 단어의 위치에 따라 쓰임새와 의미가 달라진다.
  * 임베딩 계층의 상위 계층에서 제대로 추출 및 반영되더라도 애초에 단어 임베딩할 때 고려되어 입력으로 주어진 것에 비해서 불리하다.

> 신경망의 입력 계층의 가중치 파라미터에만 적용

* 기존 적용 방법이 임베딩 계층에 국한된다.
  * 임베딩 계층 가중치 파라미터만 한정하면 전역 최소점에 더 가까울 것이다.
  * 신경망 전체 가중치 파라미터를 고려하면 최적화에 유리할 지 알 수 없다.  
  $\therefore$ 신경망 전체에 대해 사전 훈련 방법이 더 낫다.

### 15.2.3 자연어 처리에서 전이학습의 효과

* 기존의 단점을 효과적으로 보완해서 자연어 처리 문제 해결
* 자연어 처리 분야에서도 수많은 문장을 수집하여 학습한 신경망을 통해 다른 문제에 성공적으로 적용하고 성능을 개선하는 방법 소개

* 자연어 처리에서 전이학습 분류  
<img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/image6.png?raw=1" width="50%" height="50%">

* 자연어 처리에서의 사전 훈련은 다른 분야의 사전 훈련 보다 훨씬 유리한 점 갖는다.
  * 일반 문장들에 대해서 단순히 언어 모델을 훈련하는 것 만으로도 매우 큰 전이학습으로 인한 성능 향상을 볼 수 있다.
  * 일반적인 문장들의 수집은 매우 쉽고 값싸므로 전이학습이 더욱 큰 효과를 가진다.(병렬 코퍼스, 레이블링 수집 비용 대비)  
  = 기존의 데이터셋 수집에 비해 일반 코퍼스 수집은 거의 비용이 들지 않으므로, 노력 대비 매우 큰 성과를 위험 부담 없이 얻을 수 있다.

## 15.3 ELMO

* ELMO(Embedding from Language MOdel)
  * 입력으로 주어진 문장을 단방향 LSTM 넣고, 정방향 / 역방향 언어 모델을 각각 훈련
  * 언어 모델은 자기회귀 모델이므로 **양방향 LSTM으로 훈련 불가능**  
  $ p(t_1,t_2,\cdots, t_N) = \prod_{k=1}^N p(t_k | t_1, t_2, \cdots, t_{k-1}) \\ \phantom{p(t_1,t_2,\cdots, t_N)}= \prod_{k=1}^N p(t_k | t_{k+1}, t_{k+2}, \cdots, t_N)$
  * LSTM은 여러 개 계층으로 구성 가능하며 각 계층이 주어진 문장 내의 토큰 개수 만큼 정방향과 역방향에 대해 훈련된다.

* 임베딩 계층과 소프트맥스 계층은 정방향과 역방향에 대해서 공유된다.
  * 이전 레이어의 각 방향별 출력값이 같은 방향의 다음 레이어로만 전달됨에 유의하라.

* 정방향과 역방향 LSTM은 별도로 구성된다.($\vec{\theta}_{LSTM}, \overset{\leftarrow}{\theta}_{LSTM}$)
* LSTM 입력을 위한 임베딩 계층 $\theta_x$, 출력을 받아 단어 예측하는 소프트맥스 계층 $\theta_s$는 가중치 파라미터 공유

$$ \mathcal{L}(\Theta) = \sum_{k=1}^N \log p(t_k | t_1, \cdots, t_{k-1} ; \theta_x, \vec{\theta}_{LSTM}, \theta_s), \\ \text{ where } \Theta = \{\theta_x, \vec{\theta}_{LSTM}, \overset{\leftarrow}{\theta}_{LSTM}, \theta_s\}
$$

* 단어의 임베딩 벡터와 LSTM 계층별 은닉 상태를 time-step별로 얻을 수 있다.
  * 가중합을 구하여 해당 단어/토큰에 대한 문장 내 임베딩 표현인 $\text{ELMO}_k^{task}$ 구한다.

$$ \text{ELMO}_k^{task} = \gamma^{task} \sum_{j=0}^L s_j^{task} \cdot h_{k,j}^{LM} \\
\text{where } h_{k,j}^{LM} = \begin{cases}
x_k, & \mbox{if }j\mbox{ = 0} \\
[\vec{h}_{k,j}^{LM};\overset{\leftarrow}{h}_{k,j}^{LM}], & \mbox{if }j\mbox{ > 0}
\end{cases}
$$

* 표현(representation)
  * 각 time-step k 별로 L개의 LSTM 은닉 상태를 가중치 $s_j^{task}$ 에 따라 가중합을 구한다.
    * j=0 : 단어의 임베딩 벡터
    * j>0 : 정방향과 역방향의 LSTM의 은닉상태 $
[\vec{h}_{k,j}^{LM};\overset{\leftarrow}{h}_{k,j}^{LM}]$가 연계 구성

* 이 표현은 푸는 문제에 따라 달라지므로 task로 표시한다.
* ELMO 표현은 실제 문제 수행을 위한 신경망의 입력 벡터에 연계되어 쓰인다.
  * ELMO 표현을 위한 신경망 가중치 파라미터는 갱신되지 않는다.
  * 기존 언어 모델에 훈련하여 얻어진 가중치 파라미터를 고정하여 추론에 사용
  * 수식에서 각 단어의 표현
    * LSTM 정방향과 역방향 은닉상태도 포함하므로, **문맥에 따른 단어 의미 표현 가능하다.**
    * 'play' : 함께 출현한 문장 내 단어에 따라 연주하다 또는 재생하다 의미 가능하다.
  * 다양한 자연어 처리 문제에 이르기까지 큰 성능 개선을 이루었다.

* ELMo : 언어 모델을 훈련해 얻어지는 신경망의 은닉 상태를 단어의 임베딩에 사용하여 문장의 문맥에 따른 의미를 반영할 수 있었다.
  * 레이블링 필요 없는 무한히 수집 가능한 일반 문장으로 사전 훈련한다.  
  $\rightarrow$ 별도 추가 데이터셋 없이 성능 크게 개선할 수 있다는 점에서 큰 의미가 있다.